# Demos: Lecture 12

In [1]:
import pennylane as qml
import numpy as np

from lecture12_helpers import *

## Demo 1: Order finding

In [1]:
N = 13
a = 5

In [2]:
for exp in range(1, N):
    if (a ** exp) % N == 1:
        print(f"The order of {a} is {exp}")
        break

The order of 5 is 4


In [3]:
def run_order_finding(a, N):
    U_Na = get_U_Na(a, N)
    
    num_estimation_qubits = 10
    num_target_qubits = int(np.log2(len(U_Na)))
    
    estimation_wires = range(num_estimation_qubits)
    target_wires = range(num_estimation_qubits, num_estimation_qubits + num_target_qubits)
    
    dev = qml.device('default.qubit', wires=num_estimation_qubits+num_target_qubits, shots=1)
    
    @qml.qnode(dev)
    def find_order():
        # Prepare target register
        qml.PauliX(wires=target_wires[-1])
        
        # Do phase estimation
        qml.QuantumPhaseEstimation(
            U_Na,
            estimation_wires=estimation_wires,
            target_wires=target_wires
        )
        
        return qml.sample(wires=estimation_wires)

    possible_r = []
    
    for _ in range(10):
        sample = find_order()
        #print(f"Sample = {sample}")
        phase = fractional_binary_to_float(sample)
        #print(f"Numerical phase = {phase}")
        est_r = phase_to_order(phase, N)
        #print(f"Guess for r = {est_r}")
        possible_r.append(est_r)
        
    return max(possible_r)

In [4]:
run_order_finding(a, N)

NameError: name 'get_U_Na' is not defined

## Demo 2: Shor's algorithm

<img src="fig/shor-flowchart.jpeg" width="500px">

In [ ]:
def shors_algorithm(N):
    for _ in range(10):
        a = np.random.choice(list(range(2, N-1)))

        # if np.gcd(a, N) != 1:
        #     print("We got lucky!")
        #     p = np.gcd(a, N)
        #     q = N // p
        #     return p, q

        # Slightly adjusted from the live demo in class
        r = run_order_finding(a, N)

        if r % 2 == 1:
            continue
            
        x = (a ** (r // 2)) % N

        if x == 1 or x == (N - 1):
            continue
            
        p = np.gcd(x - 1, N)
        q = np.gcd(x + 1, N)
        return p, q

In [ ]:
N = 187

for _ in range(100):
    p, q = shors_algorithm(N)
    if p * q == N:
        print(f"p={p}\nq={q}")
        break